In [1]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
# from scipy.spatial.distance import mahalanobis

In [2]:
import sys, os
sys.path.insert(0, '/content/Data-Science-Project-2-2021-2-Nowcasting')

### Load raw_tuple (numpy array)

In [5]:
raw_tuple = np.load("../data/raw_tuple.npy")
raw_tuple_loc = np.load("../data/raw_tuple_loc.npy")

#### raw_tuple (n_imgs, (pixel_value, counts))

In [6]:
raw_tuple.shape

(10, 2)

In [18]:
raw_tuple

array([[   188,  84796],
       [   103,  61226],
       [   160, 521425],
       [   222, 435809],
       [   171,  24435],
       [   154,  24233],
       [   120,   6495],
       [    86,   1002],
       [    69,     17],
       [   205, 247380]], dtype=int64)

In [36]:
features = np.zeros((255,))
for key, val in raw_tuple:
  features[key] = key*val

In [43]:
mu = features/(2034*2048)
mu.shape

(255,)

#### raw_tuple_loc (?, (location_x, location_y, pixel_value, count))

In [20]:
raw_tuple_loc.shape

(899683, 4)

In [23]:
raw_tuple_loc[:2]

array([[1312,   10,  103,   10],
       [1431,   16,  103,   10]], dtype=int64)

In [33]:
features_wloc = np.zeros((2034,2048,255))

In [34]:
for loc_x, loc_y, key, val in raw_tuple_loc:
  features_wloc[loc_x][loc_y][key] = val

In [35]:
features_wloc.shape

(2034, 2048, 255)

In [45]:
mu_2dim = np.expand_dims(mu, axis=0)
mu_3dim = np.expand_dims(mu_2dim, axis=0)
mu_3dim.shape

(1, 1, 255)

In [46]:
diff = features_wloc - mu_3dim

In [54]:
diff.shape

(4165632, 255)

In [53]:
diff = diff.reshape(-1, 255)

In [51]:
gram = np.matmul(diff.T, diff)/(2034*2048-1)

In [56]:
eig_value, eig_vector = np.linalg.eigh(gram)
eig_value, eig_vector = eig_value[::-1], eig_vector[:, ::-1]

In [57]:
small_value =  1e-10
mask = eig_value > 1e-10

In [58]:
print("Non-zero eigenvalues:", mask.sum())
print("Shape of Gram Matrix:", gram.shape)

Non-zero eigenvalues: 10
Shape of Gram Matrix: (255, 255)


In [62]:
eig_vector.shape

(255,)

In [69]:
S = gram/(2034*2048-1)
S_pinv = S.T
D = np.sqrt(diff @ S_pinv @ diff.T)

MemoryError: Unable to allocate 126. TiB for an array with shape (4165632, 4165632) and data type float64